# **ETL de Google Maps**  
**Empezamos nuestro trabajo con la extracción de los datos de archivos json:**

Importamos librerias necesarias:

In [32]:
import pandas as pd
import json
import os

**Carpeta de entrada**

In [33]:
Ruta_review_estados = r"G:\Jorge\Desktop\Google Maps\reviews-estados"
Ruta_data_procesada = '../../Data/data_procesada'

**Carpeta de salida**

In [34]:
Ruta_data_preprocesada = '../../Data/data_preprocesada'
Ruta_data_procesada = '../../Data/data_procesada' # En este caso es la misma carpeta para la entrada y salida de algunos dataframes

## **Extracción de Datos:**

El siguiente código crea diferentes dataframes por cada estado, leyendo cada archivo JSON de su respectivo estado.  
Guarda cada dataframe en una lista y estos luego son concatenados en un único dataframe.

In [35]:
# Función que retorna un dataframe, dependiendo de la ruta que varia en el ciclo for de más abajo
# Abre cada directorio diferente e itera sobre cada archivo JSON:
def cargar_reviews(ruta_estado):
    reviews_estado = []
    for directorio in os.listdir(ruta_estado):
        ruta_archivo = os.path.join(ruta_estado, directorio)
        with open(ruta_archivo, 'r') as archivo:
            for linea in archivo:
                review_data = json.loads(linea)
                reviews_estado.append(review_data)
    return pd.DataFrame(reviews_estado)

Se eligieron a los estados de 'New York', 'Pensilvania', 'Florida', 'California' y 'Nevada' como muestra para el estudio por compatibildiad con los datos disponibles entre Google Maps y Yelp

In [36]:
estados = ['review-New_York', 'review-Pennsylvania', 'review-Florida', 'review-California', 'review-Nevada'] # Carpetas de estados
dfs = []    # Lista donde se guardan cada dataframe                       

# Iteramos por cada estado en la lista de estados:
for estado in estados:
    ruta_estado = f'{Ruta_review_estados}/{estado}/' # Ruta de cada estados por iteración
    df_estado = cargar_reviews(ruta_estado)                   # Se llama a la función 'cargar_reviews'
    dfs.append(df_estado)                                     # Se añade el dataframe a la lista

**Concatenamos los dataframes en uno solo:**

In [37]:
df = pd.concat(dfs, ignore_index=True)

Eliminamos las columna innecesarias: 'pics' y 'resp'

In [38]:
drops = ["pics", "resp"]
df.drop(drops, axis=1, inplace=True)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12450000 entries, 0 to 12449999
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   gmap_id  object
dtypes: int64(2), object(4)
memory usage: 569.9+ MB


### **Guardamos en un parquet los resultados:**    
**No queremos hacer devuelta todo el tiempo la lectura de json's (tarda demasiado)**

In [40]:
df.to_parquet(os.path.join(Ruta_data_preprocesada, "reviews_estado_puro.parquet"))

## **Transformación de Datos:**  

**Leemos el parquet guardado:**

In [41]:
df_reviews = pd.read_parquet(os.path.join(Ruta_data_preprocesada, "reviews_estado_puro.parquet"))

In [42]:
df_reviews.isna().sum()

user_id          0
name             0
time             0
rating           0
text       5190600
gmap_id          0
dtype: int64

Para poder filtrar las reviews de solo restaurantes, tenemos que traer ya los datos de estos mismos restaurantes, a traves del parquet:  
'restaurantes.parquet'. Para el primer paso traemos el archivo:

In [43]:
restaurantes = pd.read_parquet(os.path.join(Ruta_data_procesada, "restaurantes.parquet"))

Filtramos las reviews a traves de los restaurantes:

In [44]:
reviews_filtradas = df_reviews[df_reviews['gmap_id'].isin(restaurantes['gmap_id'])]

df_reviews_filtradas= pd.DataFrame(reviews_filtradas)

In [45]:
df_reviews_filtradas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1776939 entries, 165 to 11580817
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   gmap_id  object
dtypes: int64(2), object(4)
memory usage: 94.9+ MB


In [46]:
df_reviews_filtradas.head(5)

,user_id,name,time,rating,text,gmap_id
165,109434968607034492648,Tony Pinto,1384783434547,5,I came by yesterday to pick up some pizza on t...,0x89c258ffaeaba947:0x8355860772a595a9
166,108584493264435262292,Benji,1424212048498,1,Food was burnt and not good. You would think s...,0x89c258ffaeaba947:0x8355860772a595a9
167,108987049306954366378,Michael Rahmani,1419979427508,1,This place is horrible. I ordered two ziti piz...,0x89c258ffaeaba947:0x8355860772a595a9
168,107767094884093007779,eliau piha,1404689173836,5,Love their pizza and their service!\nREALLY IT...,0x89c258ffaeaba947:0x8355860772a595a9
169,107373453968933712290,Matt Schaffnit,1477384254400,5,Used to get lunch here nearly every day. You w...,0x89c258ffaeaba947:0x8355860772a595a9


**Fusionaremos con merge() los Dataframes: 'restaurantes' y 'df_reviews_filtradas'**

Para más claridad cambiaremos los nombres de las columnas 'name' de ambos dataframes a nombres más especificos: user_name y restaurant_name.

In [47]:
df_reviews_filtradas.rename(columns={'name':'user_name'}, inplace=True)

restaurantes.rename(columns={'name':'restaurant_name'}, inplace=True)

In [48]:
# Fusionamos:
df_reviews_filtradas = pd.merge(df_reviews_filtradas, restaurantes[['gmap_id', 'restaurant_name', 'state', 'city']], on='gmap_id', how='inner')
df_reviews_filtradas.head(5)

,user_id,user_name,time,rating,text,gmap_id,restaurant_name,state,city
0,109434968607034492648,Tony Pinto,1384783434547,5,I came by yesterday to pick up some pizza on t...,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New_York,New York
1,108584493264435262292,Benji,1424212048498,1,Food was burnt and not good. You would think s...,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New_York,New York
2,108987049306954366378,Michael Rahmani,1419979427508,1,This place is horrible. I ordered two ziti piz...,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New_York,New York
3,107767094884093007779,eliau piha,1404689173836,5,Love their pizza and their service!\nREALLY IT...,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New_York,New York
4,107373453968933712290,Matt Schaffnit,1477384254400,5,Used to get lunch here nearly every day. You w...,0x89c258ffaeaba947:0x8355860772a595a9,Raffaello Kosher Pizza,New_York,New York


In [49]:
df_reviews_filtradas.isna().sum()

user_id                 0
user_name               0
time                    0
rating                  0
text               692864
gmap_id                 0
restaurant_name         0
state                   0
city                    0
dtype: int64

Como observamos, los únicos nulos que posee nuestro Dataframe son el texto de las reviews, por lo tanto los rellenaremos con 'SD' (Sin Dato):

In [50]:
df_reviews_filtradas["text"] = df_reviews_filtradas["text"].fillna("SD")
df_reviews_filtradas.isna().sum()

user_id            0
user_name          0
time               0
rating             0
text               0
gmap_id            0
restaurant_name    0
state              0
city               0
dtype: int64

Reordenamos las columnas:

In [51]:
orden = ["user_id",	"user_name", "time", "rating", "text", "restaurant_name", "state", "city", "gmap_id"]

df_reviews_filtradas = df_reviews_filtradas[orden]
df_reviews_filtradas.head(2)

,user_id,user_name,time,rating,text,restaurant_name,state,city,gmap_id
0,109434968607034492648,Tony Pinto,1384783434547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
1,108584493264435262292,Benji,1424212048498,1,Food was burnt and not good. You would think s...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


**Convertimos a datetime la columna 'time'**  
Esta se encontraba en formato unix.

In [52]:
# Creamos una copia para trabajar con seguridad:
df_copy = df_reviews_filtradas.copy()

df_copy["time"] = pd.to_datetime(df_reviews_filtradas["time"], unit="ms")
df_copy.head()

,user_id,user_name,time,rating,text,restaurant_name,state,city,gmap_id
0,109434968607034492648,Tony Pinto,2013-11-18 14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
1,108584493264435262292,Benji,2015-02-17 22:27:28.498,1,Food was burnt and not good. You would think s...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
2,108987049306954366378,Michael Rahmani,2014-12-30 22:43:47.508,1,This place is horrible. I ordered two ziti piz...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
3,107767094884093007779,eliau piha,2014-07-06 23:26:13.836,5,Love their pizza and their service!\nREALLY IT...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
4,107373453968933712290,Matt Schaffnit,2016-10-25 08:30:54.400,5,Used to get lunch here nearly every day. You w...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


Ahora regularizamos los datos de la columna 'time' y ordenamos:

In [53]:
df_copy['year'] = df_copy['time'].dt.year
df_copy['month'] = df_copy['time'].dt.month
df_copy['day'] = df_copy['time'].dt.day
df_copy['hour'] = df_copy['time'].dt.strftime('%H:%M:%S.%f').str.rstrip('0')

orden_dos = ["user_id",	"user_name", "time", "year", "month", "day", "hour", "rating", "text", "restaurant_name", "state", "city", "gmap_id"]
df_copy = df_copy[orden_dos]
df_copy.head(3)

,user_id,user_name,time,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
0,109434968607034492648,Tony Pinto,2013-11-18 14:03:54.547,2013,11,18,14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
1,108584493264435262292,Benji,2015-02-17 22:27:28.498,2015,2,17,22:27:28.498,1,Food was burnt and not good. You would think s...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9
2,108987049306954366378,Michael Rahmani,2014-12-30 22:43:47.508,2014,12,30,22:43:47.508,1,This place is horrible. I ordered two ziti piz...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


Eliminamos la columna 'time':

In [54]:
df_copy.drop("time", axis=1, inplace=True)

df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776939 entries, 0 to 1776938
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          object
 1   user_name        object
 2   year             int32 
 3   month            int32 
 4   day              int32 
 5   hour             object
 6   rating           int64 
 7   text             object
 8   restaurant_name  object
 9   state            object
 10  city             object
 11  gmap_id          object
dtypes: int32(3), int64(1), object(8)
memory usage: 142.3+ MB


**Verificación de Datos Duplicados:**

In [55]:
# Contar la cantidad de filas duplicadas
cantidad_duplicados = df_copy.duplicated().sum()
print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 71408


Observamos duplicados:

In [56]:
df_copy[df_copy.duplicated()].head(5)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
328,103349920500088982390,hamdy mustafa,2021,5,15,13:23:53.554,5,The pastrami was really really good. Service w...,The Pastrami Shoppe,New_York,Staten Island,0x89c24b375e7555c5:0xfc278236316bdf0b
333,100316639659616282316,David Lambert,2021,6,19,22:30:44.268,1,They are way overpriced for the size of the sa...,The Pastrami Shoppe,New_York,Staten Island,0x89c24b375e7555c5:0xfc278236316bdf0b
354,109004986980114616349,Merri Silverstein,2007,8,1,00:00:00.,4,"see and be seen!! summertime fun, maybe after ...",Baraonda,New_York,New York,0x89c258c07bb81027:0xbe685167f2b5bc33
448,110076291766488179945,k brenen,2018,7,27,17:19:34.581,4,So impressed for food court food. First off th...,Mumbles Chicken and Waffles,New_York,Cheektowaga,0x89d30df8ef5a1d7d:0x584a55b87c0887a3
465,109434968607034492648,Tony Pinto,2013,11,18,14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


In [57]:
# Vemos un caso en específico:
df_copy[df_copy["user_id"] == '117440349723823658676']

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
1168636,117440349723823658676,Anthony Kim,2019,3,7,05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1169316,117440349723823658676,Anthony Kim,2019,3,7,05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [58]:
# Eliminar las filas duplicadas
df_sin_duplicados = df_copy.drop_duplicates()

# Contar la cantidad de filas duplicadas nuevamente:
cantidad_duplicados = df_sin_duplicados.duplicated().sum()
print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 0


**Recuento y Promedio de Reseñas:**   
Los datos se agrupan por user_id y user_name para calcular el número total de reseñas por usuario y su calificación promedio.  
Este análisis centrado en el usuario puede revelar patrones en el comportamiento de las reseñas y el compromiso del usuario.

In [59]:
# Contar el número de reseñas por usuario
user_review_counts = df_sin_duplicados.groupby(['user_id', 'user_name']).size().reset_index(name='review_count')

# Calcular la calificación promedio por usuario
user_calif_promedio = df_sin_duplicados.groupby(['user_id', 'user_name'])['rating'].mean().reset_index()
user_calif_promedio.rename(columns={'rating': 'average_rating'}, inplace=True)

# Combinar los recuentos de reseñas y las calificaciones promedio por usuario en un solo DataFrame
df_user_review_counts_ord = pd.merge(user_review_counts, user_calif_promedio, on=['user_id', 'user_name'], how='inner')
df_user_review_counts_ord = df_user_review_counts_ord.sort_values(by='review_count', ascending=False)

# Mostrar las primeras filas del DataFrame ordenado
df_user_review_counts_ord.head()

,user_id,user_name,review_count,average_rating
291440,104819208193648646391,Gregor J. Rothfuss,102,3.941176
405829,106654503918907830147,The Corcoran Group,91,4.054945
918439,114955250538652050870,Javier Kohen,56,3.964286
926238,115082761597075271038,ej shortell,45,4.111111
592082,109673791694826464177,Jackie Gordon Singing Chef,44,3.863636


## **Carga de Datos:**  

Guardaremos nuestros dataframes en archivos parquet:

Datos de las reviews, Dataframe 'df_copy'. Lo usaremos para análisis.

In [60]:
df_copy.to_parquet(os.path.join(Ruta_data_procesada, "reviews_google_maps.parquet"), index=False)

Datos de los usuarios, Dataframe 'df_user_review_counts_ord'. Lo usaremos para análisis.

In [61]:
df_user_review_counts_ord.to_parquet(os.path.join(Ruta_data_procesada, "users_google_maps.parquet"), index=False)